In [ ]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [ ]:
from selenium import webdriver


driver = webdriver.Chrome()


driver.get("https://agilgestion.com.ar/")

login_button = driver.find_element(By.ID, "login-button")
login_button.click()


element = driver.find_element(By.ID, "user")
element.send_keys("****")
element = driver.find_element(By.NAME, "password")
element.send_keys("****")
element.submit()

elements_with_nav_link_class = driver.find_elements(By.CLASS_NAME, 'nav-link')


informes_link = next(element for element in elements_with_nav_link_class if "Informes" in element.text)

informes_link.click()

ventas_button = driver.find_element(By.XPATH, '//button[@class="dropdown-item dropdown-toggle" and @type="submit" and @data-toggle="dropdown" and text()="Compras"]')
ventas_button.click()

detalles_items_link = driver.find_element(By.XPATH, '//a[@class="dropdown-item" and text()="Items por Centro de Costos"]')
detalles_items_link.click()

fecha_input = driver.find_element(By.ID, 'filtros_desde_fecha')

fecha_input.clear()

fecha_input.send_keys('01/01/2023')

fecha_input = driver.find_element(By.ID, 'filtros_hasta_fecha')

fecha_input.clear()

fecha_input.send_keys('20/12/2023')

select_element = driver.find_element(By.ID, "filtros_incluye_sin_especificar")

select = Select(select_element)

select.select_by_value("true")


generar_button = driver.find_element(By.ID, 'informe_generar')
generar_button.click()


descargar_excel_span = driver.find_element(By.XPATH, '//span[text()="Descargar Excel"]')
descargar_excel_span.click()

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd


file_path = r"C:\Users\lenov\Music\PYTHON SELENIUM\items_detalles_por_centro_costo.xls"
df = pd.read_excel(file_path)
df.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
pip install google-auth

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install google-api-python-client


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow


In [ ]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
import datetime


def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

In [ ]:
from googleapiclient.http import MediaFileUpload


CLIENT_SECRET_FILE = "credentials.json"
API_NAME = "drive"
API_VERSION = "v3"
SCOPES = ["https://www.googleapis.com/auth/drive"]

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)


def upload_file(service, file_path, file_name, folder_id=None):
    media = MediaFileUpload(file_path, resumable=True)
    body = {
        'name': file_name,
        'mimeType': 'application/octet-stream'
    }
    if folder_id:
        body['parents'] = [folder_id]

    request = service.files().create(body=body, media_body=media)
    response = None
    while response is None:
        status, response = request.next_chunk()
        if status:
            print(f'Uploaded {int(status.progress() * 100)}%')
    print(f'File {file_name} uploaded successfully.')


file_path = r"C:\Users\lenov\Music\PYTHON SELENIUM\items_detalles_por_centro_costo.xls"

file_name = "items_detalles_por_centro_costo"

folder_id = "1PbbmY3UqhIM6HSauHUy1fR5-qPNjbcSq"

upload_file(service, file_path, file_name, folder_id)




credentials.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully
File items_detalles_por_centro_costo uploaded successfully.
